In [1]:
import sys
import os
sys.path.append(os.path.join(os.path.abspath(os.path.dirname("__file__")), '..'))
from utils import data_handling, model_development
import matplotlib.pyplot as plt
from ipywidgets import interact

In [2]:
##
# Configure the model training policy:
#
model_development.configure_training_policy()

In [3]:
##
# Load and preprocess data:
#
cell_images, cell_labels, cell_types = data_handling.load_data_from_file('../data/labels.csv', True)
print([cell_images.shape, len(cell_labels), len(cell_types)])

@interact(n = (0, cell_images.shape[0] - 1))
def display_data(n = 0):
    fig, axs = plt.subplots()
    axs.imshow(cell_images[n], cmap = 'gray')
    axs.set_title(f'Class: {cell_labels[n]} - Type: {cell_types[n]}')
    return None

[(2624, 300, 300), 2624, 2624]


interactive(children=(IntSlider(value=0, description='n', max=2623), Output()), _dom_classes=('widget-interact…

In [4]:
##
# Prepare data for model training:
#
train_test_split = int(0.85 * cell_images.shape[0])
X_train, Y_train, Z_train  = data_handling.preprocess_data(\
    cell_images[: train_test_split], 
    cell_labels[: train_test_split], 
    cell_types[: train_test_split])
X_test, Y_test, Z_test = \
    cell_images[train_test_split : ], \
    cell_labels[train_test_split : ], cell_types[train_test_split : ]

print([X_train.shape, len(Y_train), len(Z_train)])
print([X_test.shape, len(Y_test), len(Z_test)])

[(2230, 300, 300, 3), 2230, 2230]
[(394, 300, 300), 394, 394]


In [5]:
##
# Train classification model (first phase):
#
optimizer = model_development.create_optimizer('nadam')
vgg19 = \
    model_development.vgg19(input_shape = (300, 300, 3), weights = 'imagenet', freeze_convolutional_base = True, display_model_information = False)

history, training_time = \
    model_development.train_classification_model(training_phase = 1, model = vgg19, 
    optimizer = optimizer, training_metrics = ['accuracy', 'Precision', 'Recall'],
    model_name = 'VGG19', version = '20221003_fs', 
    X = X_train, Y = Y_train, metric_to_monitor = 'val_accuracy', 
    no_of_epochs = 200, batch_size = 16, validation_split_ratio = 0.15)

Epoch 1/200
119/119 [==============================] - ETA: 0s - loss: 0.8731 - accuracy: 0.5235 - precision: 0.4832 - recall: 0.4089
Epoch 1: val_accuracy improved from -inf to 0.57015, saving model to ../models/weights\VGG19_20221003_fs.hdf5
119/119 [==============================] - 23s 146ms/step - loss: 0.8731 - accuracy: 0.5235 - precision: 0.4832 - recall: 0.4089 - val_loss: 0.6899 - val_accuracy: 0.5701 - val_precision: 1.0000 - val_recall: 0.0137
Epoch 2/200
119/119 [==============================] - ETA: 0s - loss: 0.6526 - accuracy: 0.6301 - precision: 0.6848 - recall: 0.3736
Epoch 2: val_accuracy improved from 0.57015 to 0.58209, saving model to ../models/weights\VGG19_20221003_fs.hdf5
119/119 [==============================] - 16s 135ms/step - loss: 0.6526 - accuracy: 0.6301 - precision: 0.6848 - recall: 0.3736 - val_loss: 0.7878 - val_accuracy: 0.5821 - val_precision: 0.8750 - val_recall: 0.0479
Epoch 3/200
119/119 [==============================] - ETA: 0s - loss: 0.6361

In [7]:
##
# Train classification model (second phase):
#
optimizer = model_development.create_optimizer('adam')
history, training_time = \
    model_development.train_classification_model(training_phase = 2, 
    model = '../models/weights/VGG19_20221003_fs.hdf5', 
    optimizer = optimizer, training_metrics = ['accuracy', 'Precision', 'Recall'],
    model_name = 'VGG19', version = '20221003_ss', 
    X = X_train, Y = Y_train, metric_to_monitor = 'val_accuracy', 
    no_of_epochs = 200, batch_size = 16, validation_split_ratio = 0.15)

Epoch 1/200
119/119 [==============================] - ETA: 0s - loss: 0.6501 - accuracy: 0.6470 - precision: 0.6805 - recall: 0.4487
Epoch 1: val_accuracy improved from -inf to 0.66866, saving model to ../models/weights\VGG19_20221003_ss.hdf5
119/119 [==============================] - 28s 217ms/step - loss: 0.6501 - accuracy: 0.6470 - precision: 0.6805 - recall: 0.4487 - val_loss: 0.6585 - val_accuracy: 0.6687 - val_precision: 0.7215 - val_recall: 0.3904
Epoch 2/200
119/119 [==============================] - ETA: 0s - loss: 0.5781 - accuracy: 0.6897 - precision: 0.7158 - recall: 0.5478
Epoch 2: val_accuracy improved from 0.66866 to 0.68358, saving model to ../models/weights\VGG19_20221003_ss.hdf5
119/119 [==============================] - 25s 212ms/step - loss: 0.5781 - accuracy: 0.6897 - precision: 0.7158 - recall: 0.5478 - val_loss: 0.5922 - val_accuracy: 0.6836 - val_precision: 0.7222 - val_recall: 0.4452
Epoch 3/200
119/119 [==============================] - ETA: 0s - loss: 0.5665